In [ ]:
import torch
import multiprocessing
cores = multiprocessing.cpu_count()
cores

4

In [ ]:
import os
from tqdm import tqdm_notebook
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
from sklearn import utils
DIR = '/content/drive/My Drive/Uniprot/'

In [ ]:
train_tagged_file = os.path.join(DIR, 'tagged_data/sequences/train/train.pkl')
test_tagged_file = os.path.join(DIR, 'tagged_data/sequences/test/test.pkl')

In [ ]:
from tqdm import tqdm_notebook
model_dbow = Doc2Vec(dm=0, vector_size=512, window=32, negative=4, min_count=2, workers=cores, alpha=0.08, min_alpha=0.001, hs=0)
model_dbow.build_vocab([x for x in tqdm_notebook(train_tagged)])
model_dmm = Doc2Vec(dm=1, vector_size=512, window=32, negative=4, min_count=2, workers=cores, alpha=0.08, min_alpha=0.001, hs=0, dm_mean=0)
model_dmm.build_vocab([x for x in tqdm_notebook(train_tagged)])
model_dmc = Doc2Vec(dm=1, dm_concat=0, vector_size=512, window=32, negative=4, min_count=2, workers=cores, alpha=0.08, min_alpha=0.001)
model_dmc.build_vocab([x for x in tqdm_notebook(train_tagged)])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
tokens = [x for x in train_tagged.values]
for epoch in tqdm_notebook(range(10)):
    model_dbow.train(utils.shuffle(tokens), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    # model_dmm.train(utils.shuffle(tokens), total_examples=len(train_tagged.values), epochs=1)
    # model_dmm.alpha -= 0.002
    # model_dmc.train(utils.shuffle(tokens), total_examples=len(train_tagged.values), epochs=1)
    # model_dmc.alpha -= 0.002

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
model_dbow.save(DIR + 'tagged_data/sequences/dbow/model/model_dbow')
# model_dmm.save(DIR + 'tagged_data/sequences/dmm/model/model_dmm')
# model_dmc.save(DIR + 'tagged_data/sequences/dmc/model/model_dmc')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, epochs=20)) for doc in sents])
    return targets, regressors

In [ ]:
train_tagged = pd.read_pickle(train_tagged_file)
test_tagged = pd.read_pickle(test_tagged_file)
# print('[TRAIN_LOADED]')
import pickle
# ctr = 33
# for i in tqdm_notebook(range(33*10000, len(train_tagged), 10000)):
#     y_train, X_train = vec_for_learning(model_dbow, train_tagged[i:i+10000])
#     ctr += 1
#     with open(DIR + 'tagged_data/annotations/dbow/train/regressors' + str(ctr) + '.pkl', 'wb') as fp:
#         pickle.dump(X_train, fp)
#     with open(DIR + 'tagged_data/annotations/dbow/train/targets' + str(ctr) + '.pkl', 'wb') as fp:
#         pickle.dump(y_train, fp)
#     print(f'[TRAIN-{ctr}-COMPLETE]')
def get_vectors(model, train_tagged, name, training=True):
    y_train, X_train = vec_for_learning(model, train_tagged)
    if training:
        dir = 'train'
    else:
        dir = 'test'
    with open(DIR + 'tagged_data/sequences/' + name + '/' + dir + '/regressors.pkl', 'wb') as fp:
        pickle.dump(X_train, fp)
    with open(DIR + 'tagged_data/sequences/' + name + '/' + dir + '/targets.pkl', 'wb') as fp:
        pickle.dump(y_train, fp)
    print(f'[TRAIN-' + name + '-' + dir +  '-COMPLETE]')
get_vectors(model_dbow, train_tagged, 'dbow', True)
get_vectors(model_dbow, test_tagged, 'dbow', False)
# get_vectors(model_dmm, train_tagged, 'dmm', True)
# get_vectors(model_dmm, test_tagged, 'dmm', False)
# get_vectors(model_dmc, train_tagged, 'dmc', True)
# get_vectors(model_dmc, test_tagged, 'dmc', False)

[TRAIN-dbow-train-COMPLETE]
[TRAIN-dbow-test-COMPLETE]


In [ ]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

def get_contact_three_vectors(model1, model2, model3, corpus, size)
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in tqdm_notebook(corpus.index):
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix],model3.docvecs[prefix])
        n += 1
    return vecs


X_train_dbow_dmm = get_concat_vectors(model_dbow,model_dmm, X_train, 512)
X_train_dbow_dmm = get_concat_vectors(model_dbow,model_dmm, X_test, 512)
print('[DBOW+DMM] -- Concatenated')

with open(DIR + 'tagged_data/sequences/dbow+dmm/train/regressors.pkl', 'wb') as fp:
    pickle.dump(X_train_dbow_dmm, fp)
with open(DIR + 'tagged_data/sequences/dbow+dmm/test/regressors.pkl', 'wb') as fp:
    pickle.dump(X_test_dbow_dmm, fp)
print('[DBOW+DMM] -- Dumped')


X_train_dmm_dmc = get_concat_vectors(model_dmm,model_dmc, X_train, 512)
X_train_dmm_dmc = get_concat_vectors(model_dmm,model_dmc, X_test, 512)
print('[DMM+DMC] -- Concatenated')

with open(DIR + 'tagged_data/sequences/dmm+dmc/train/regressors.pkl', 'wb') as fp:
    pickle.dump(X_train_dmm_dmc, fp)
with open(DIR + 'tagged_data/sequences/dmm+dmc/test/regressors.pkl', 'wb') as fp:
    pickle.dump(X_test_dmm_dmc, fp)
print('[DMM+DMC] -- Dumped')


X_train_dbow_dmc = get_concat_vectors(model_dbow,model_dmc, X_train, 512)
X_train_dbow_dmc = get_concat_vectors(model_dbow,model_dmc, X_test, 512)
print('[DBOW+DMC] -- Concatenated')

with open(DIR + 'tagged_data/sequences/dbow+dmc/train/regressors.pkl', 'wb') as fp:
    pickle.dump(X_train_dbow_dmc, fp)
with open(DIR + 'tagged_data/sequences/dbow+dmc/test/regressors.pkl', 'wb') as fp:
    pickle.dump(X_test_dbow_dmc, fp)
print('[DBOW+DMC] -- Dumped')


X_train_dbow_dmm_dmc = get_concat_vectors(model_dbow,model_dmm,model_dmc, X_train, 512)
X_train_dbow_dmm_dmc = get_concat_vectors(model_dbow,model_dmm,model_dmc, X_test, 512)
print('[DBOW+DMM+DMC] -- Concatenated')

with open(DIR + 'tagged_data/sequences/dbow+dmm+dmc/train/regressors.pkl', 'wb') as fp:
    pickle.dump(X_train_dbow_dmm_dmc, fp)
with open(DIR + 'tagged_data/sequences/dbow+dmm+dmc/test/regressors.pkl', 'wb') as fp:
    pickle.dump(X_test_dbow_dmm_dmc, fp)
print('[DBOW+DMM+DMC] -- Dumped')

In [ ]:
import pickle
with open(DIR + 'tagged_data/sequences/dbow/test/regressors.pkl', 'rb') as fp:
    X_test = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/test/targets.pkl', 'rb') as fp:
    y_test = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/train/regressors.pkl', 'rb') as fp:
    X_train = pickle.load(fp)    
with open(DIR + 'tagged_data/sequences/dbow/train/targets.pkl', 'rb') as fp:
    y_train = pickle.load(fp)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
scaler = MinMaxScaler()
from tqdm import tqdm_notebook
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
print(len(X_train),len(y_train), len(X_test), len(y_test))

50572 50572 21674 21674


In [ ]:
from sklearn.model_selection import cross_val_score
neighbors = list(range(2, 30))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in tqdm_notebook(neighbors):
    model = knn(n_neighbors=k, weights='distance', algorithm='auto', metric='euclidean', n_jobs=-1)
    print('[SKELETON_FORMED]')
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy', verbose=6)
    print(scores.mean())
    cv_scores.append(scores.mean())

In [ ]:
import pickle

from sklearn.neighbors import KNeighborsClassifier as knn
from tqdm import tqdm_notebook
import random
model = knn(n_neighbors= 7, weights='distance', algorithm='auto', metric='euclidean', n_jobs=-1)
print('[MODEL CREATED]')
model.fit(X_train1, y_train1)
print('[TRAINING COMPLETE]')
# with open(DIR + 'tagged_data/sequences/knn/model_dbow', 'wb') as fp:
#     pickle.dump(model, fp)
# print('[MODEL DUMPED]')

[MODEL CREATED]
[TRAINING COMPLETE]


In [ ]:
from sklearn.metrics import accuracy_score
import time
idxs = [i for i in range(len(X_test1))]
total = 0
# for i in tqdm_notebook(range(10)):
#     correct = 0
#     for  j in range(101):
#         idx = random.choice(idxs)
#         if model.predict([X_test1[idx]]) == y_test1[idx]:
#             correct += 1
#     accuracy = round(correct/101, 4)
#     print('[ACCURACY] - ', accuracy)
#     total += accuracy
# print('Average Testing Accuracy: ', total/10)
start = time.time()
y_pred = model.predict(x2)
accuracy = accuracy_score(y_test1, y_pred)
print('Testing Accuracy: ' ,accuracy)
print((time.time()-start)/60, ' minutes')

Testing Accuracy:  0.9439446366782007
0.6532823363939921  minutes


In [ ]:
idxs = [i for i in  range(len(X_train))]
total = 0
for i in tqdm_notebook(range(10)):
    correct = 0
    for  j in range(101):
        idx = random.choice(idxs)
        if model.predict([X_train[idx]]) == y_train[idx]:
            correct += 1
    accuracy = round(correct/101, 4)
    print('[ACCURACY] - ', accuracy)
    total += accuracy
print('Average Training Accuracy: ', total/10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0
[ACCURACY] -  1.0

Average Training Accuracy:  1.0


In [ ]:
# train a doc2vec model

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier as knn
from tqdm import tqdm_notebook
import random
import pandas as pd

train_tagged_file = DIR + 'tagged_data/sequences/train/train.pkl'
test_tagged_file = DIR + 'tagged_data/sequences/test/test.pkl'
train_tagged = pd.read_pickle(train_tagged_file)
test_tagged = pd.read_pickle(test_tagged_file)
complete_data = pd.concat([train_tagged, test_tagged])
import pickle
# with open (DIR+'tagged_data/sequences/complete_data/data.pkl', 'wb') as fp:
#     pickle.dump(data, fp)
complete_data = complete_data.sample(frac=1)
temp = []
for entry in complete_data:
    if entry[1][0].endswith('\n'):
        tag = [entry[1][0][:-1]]
    else:
        tag = entry[1]
    temp.append(TaggedDocument([x for x in entry[0]], tag))
complete_data = pd.Series(temp)
freq = {}

print("[CREATING FREQ TABLE]")
for entry in complete_data.values:
    if entry[1][0] in freq.keys():
        freq[entry[1][0]] += 1
    else:
        freq[entry[1][0]] = 1
# x = 1/0

top_freq = {}
for k, v in freq.items():
    if v >= 1000:
        top_freq[k] = v
print(len(list(top_freq.keys())))
data = []
with open(DIR + 'tagged_data/sequences/dbow/top_freq1.pkl', 'wb') as fp:
    pickle.dump(top_freq, fp)
# stopper = 1/0

print("[FILTERING LESS FREQUENT]")
for entry in complete_data.values:
    if entry[1][0] in top_freq.keys():
        data.append(entry)

temp = {}
for entry in data:
    if entry[1][0] in temp.keys():
        temp[entry[1][0]].append(entry)
    else:
        temp[entry[1][0]] = []
print("List of families - ", temp.keys())
train_tagged = []
test_tagged = []

for v in temp.values():
    train_tagged = train_tagged + v[:int(0.4*len(v))]
    test_tagged = test_tagged + v[int(0.4*len(v)):]
print(train_tagged[0][1], train_tagged[-1][1])

# train_tagged = [TaggedDocument([str(hash(x)%1024) for x in entry[0]], entry[1]) for entry in train_tagged]
# test_tagged = [TaggedDocument([str(hash(x)%1024) for x in entry[0]], entry[1]) for entry in test_tagged]
# temp = []
# for entry in train_tagged:
#     temp.append(TaggedDocument([x for x in entry[0]], entry[1]))
# train_tagged = temp


# temp = []
# for entry in test_tagged:
#     temp.append(TaggedDocument([x for x in entry[0]], entry[1]))
# test_tagged = temp


print(train_tagged[0], test_tagged[0])
train_tagged = pd.Series(train_tagged)
test_tagged = pd.Series(test_tagged)

# with open(DIR + 'tagged_data/sequences/train/train_tagged.pkl', 'wb') as pkl:
#     pickle.dump(train_tagged, pkl)
# with open(DIR + 'tagged_data/sequences/test/test_tagged.pkl', 'wb') as pkl:
#     pickle.dump(test_tagged, pkl)
# print('[NEW DATA DUMP]')

# tt = 1/0
tokens = [x for x in train_tagged.values]

print(["MODEL CREATION"])
model_dbow = Doc2Vec(dm=0, vector_size=1024, window=32, min_count=2, workers=cores, alpha=0.08, min_alpha=0.001, hs=1)
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=1024, window=32, min_count=2, workers=cores, alpha=0.08, min_alpha=0.001)
model_dbow.build_vocab([x for x in train_tagged])
model_dmm.build_vocab([x for x in train_tagged])
print('[Model Build Initialized]')

print("[MODEL TRAINING START]")
for epoch in tqdm_notebook(range(16)):
    model_dbow.train(utils.shuffle(tokens), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.004
    model_dmm.train(utils.shuffle(tokens), total_examples=len(train_tagged.values), epochs=1)
    model_dmm.alpha -= 0.004
print('[MODEL TRAINED]')

[CREATING FREQ TABLE]
36
[FILTERING LESS FREQUENT]
List of families -  dict_keys(['chaperonin (HSP60) family', 'thiolase-like superfamily', 'radical SAM superfamily', 'metallo-dependent hydrolases superfamily', 'HisA/HisF family', 'protein kinase superfamily', 'class-I aminoacyl-tRNA synthetase family', 'class-II aminoacyl-tRNA synthetase family', 'class I-like SAM-binding methyltransferase superfamily', 'methylthiotransferase family', 'cytochrome b family', 'methyltransferase superfamily', 'TRAFAC class translation factor GTPase superfamily', 'G-protein coupled receptor 1 family', 'universal ribosomal protein uL16 family', 'transferase hexapeptide repeat family', 'MurCDEF family', 'ABC transporter superfamily', 'TRAFAC class TrmE-Era-EngA-EngB-Septin-like GTPase superfamily', 'universal ribosomal protein uL2 family', 'DEAD box helicase family', 'globin family', 'universal ribosomal protein uS12 family', 'universal ribosomal protein uS8 family', 'universal ribosomal protein uL14 family

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



[MODEL TRAINED]


In [ ]:
print(len(data), len(train_tagged), len(test_tagged))

72246 28872 43338


In [ ]:
import pickle
model_dbow.save(DIR + 'tagged_data/sequences/dbow/model1')
model_dmm.save(DIR + 'tagged_data/sequences/dmm/model1')
print('[MODELS SAVED]')
# with open(DIR + 'tagged_data/sequences/train/train_tagged_hash.pkl', 'rb') as pkl:
#     train_tagged = pickle.load(pkl)
# with open(DIR + 'tagged_data/sequences/test/test_tagged_hash.pkl', 'rb') as pkl:
#     test_tagged = pickle.load(pkl)

[MODELS SAVED]


In [ ]:
import pickle
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import threading


def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, epochs=20)) for doc in tqdm_notebook(sents)])
    return targets, regressors




def get_vectors(model, train_tagged):
    y, X = vec_for_learning(model, train_tagged)
    return y, X

y, X = get_vectors(model_dbow, train_tagged)

with open(DIR + 'tagged_data/sequences/dbow/train/regressors1.pkl', 'wb') as fp:
    pickle.dump(X, fp)
with open(DIR + 'tagged_data/sequences/dbow/train/targets1.pkl', 'wb') as fp:
    pickle.dump(y, fp)

y, X = get_vectors(model_dbow, test_tagged)

with open(DIR + 'tagged_data/sequences/dbow/test/regressors1.pkl', 'wb') as fp:
    pickle.dump(X, fp)
with open(DIR + 'tagged_data/sequences/dbow/test/targets1.pkl', 'wb') as fp:
    pickle.dump(y, fp)



y, X = get_vectors(model_dmm, train_tagged)

with open(DIR + 'tagged_data/sequences/dmm/train/regressors1.pkl', 'wb') as fp:
    pickle.dump(X, fp)
with open(DIR + 'tagged_data/sequences/dmm/train/targets1.pkl', 'wb') as fp:
    pickle.dump(y, fp)

y, X = get_vectors(model_dmm, test_tagged)

with open(DIR + 'tagged_data/sequences/dmm/test/regressors1.pkl', 'wb') as fp:
    pickle.dump(X, fp)
with open(DIR + 'tagged_data/sequences/dmm/test/targets1.pkl', 'wb') as fp:
    pickle.dump(y, fp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
x = '/content/drive/My Drive/Uniprot/tagged_data/sequences/complete_dbow/regressor_data.pkl'
x1 = DIR + 'tagged_data/sequences/complete_dbow/regressor_data.pkl'
print(x, '\n',x1)

/content/drive/My Drive/Uniprot/tagged_data/sequences/complete_dbow/regressor_data.pkl 
 /content/drive/My Drive/Uniprot/tagged_data/sequences/complete_dbow/regressor_data.pkl


In [ ]:
print(type(data_dbow), type(data_dmm))
torch.save(data_dbow, DIR + 'tagged_data/sequences/complete_dbow/data_dbow.pt')
torch.save(data_dmm, DIR + 'tagged_data/sequences/complete_dmm/data_dmm.pt')

<class 'list'> <class 'list'>


In [ ]:
with open(DIR + 'tagged_data/sequences/complete_dmm/data_dmm.pkl', 'wb') as fp:
    pickle.dump(data_dmm, fp)
with open(DIR + 'tagged_data/sequences/complete_dbow/data_dbow.pkl', 'wb') as fp:
    pickle.dump(data_dbow, fp)

In [ ]:
# get vectors of the sequences from dbow model

y_train, X_train = get_vectors(model_dmm, train_tagged, 'dbow', True)
print(['TRAIN VECTORS OBTAINED'])

y_test, X_test = get_vectors(model_dmm, test_tagged, 'dbow', False)
print(['TEST VECTORS OBTAINED'])

y_rest, X_rest = get_vectors(model_dmm, rest, 'dbow', False)
print(['REST VECTORS OBTAINED'])

['TRAIN VECTORS OBTAINED']
['TEST VECTORS OBTAINED']
['REST VECTORS OBTAINED']


In [ ]:
print(len(X_train), len(X_test), len(X_rest))
# if len(X_train < len(X_test)):
#     X_test = x_rest.values

50572 14449 28899


In [ ]:
import pickle

model_dbow.save(DIR + 'tagged_data/sequences/dmm/model/model_dmm')
with open(DIR + 'tagged_data/sequences/dmm/test/regressors.pkl', 'wb') as fp:
    pickle.dump(X_test, fp)
with open(DIR + 'tagged_data/sequences/dmm/test/targets.pkl', 'wb') as fp:
    pickle.dump(y_test, fp)
with open(DIR + 'tagged_data/sequences/dmm/train/regressors.pkl', 'wb') as fp:
    pickle.dump(X_train, fp)
with open(DIR + 'tagged_data/sequences/dmm/train/targets.pkl', 'wb') as fp:
    pickle.dump(y_train, fp)
with open(DIR + 'tagged_data/sequences/dmm/rest/regressors.pkl', 'wb') as fp:
    pickle.dump(X_rest, fp)
with open(DIR + 'tagged_data/sequences/dmm/rest/targets.pkl', 'wb') as fp:
    pickle.dump(y_rest, fp)

In [ ]:
import pickle
with open(DIR + 'tagged_data/sequences/dbow/test/regressors.pkl', 'rb') as fp:
    X_test = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/test/targets.pkl', 'rb') as fp:
    y_test = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/train/regressors.pkl', 'rb') as fp:
    X_train = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/train/targets.pkl', 'rb') as fp:
    y_train = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/rest/regressors.pkl', 'rb') as fp:
    X_rest = pickle.load(fp)
with open(DIR + 'tagged_data/sequences/dbow/rest/targets.pkl', 'rb') as fp:
    y_rest = pickle.load(fp)